<a href="https://colab.research.google.com/github/xiaoyi-yuxiao/python-code-for-FinancialAnalysis-book/blob/master/2020_06_15_chap17_runtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests_html

In [ ]:
pip install yahoo_fin

In [ ]:
pip install requests

In [ ]:
pip install bs4

In [ ]:
import numpy as np
import scipy.stats as st
from yahoo_fin.stock_info import*
from tabulate import tabulate
import requests
from bs4 import BeautifulSoup
import datetime as dt#get current date
from google.colab import files

In [ ]:
def run_test(data, v, alpha = 0.05):
  #get positive ret and neg ret
  mask = data>v
  p = mask == True
  n = mask == False
  change_sin = np.logical_xor(p[:-1],p[1:])
  sum_run = sum(change_sin)+1
  cum_sum = np.cumsum(change_sin)
  cum_sum = np.insert(cum_sum,0,0)
  #sum of positive and negative num
  n_p = sum(p)
  n_n = sum(n)

  tmp = 2 * n_p * n_n 
  tmps = n_p + n_n

  # Expected value
  r_hat = np.float64(tmp) / tmps + 1
  # Variance
  s_r_squared = (tmp*(tmp - tmps)) / (tmps*tmps*(tmps-1))
  # Standard deviation
  s_r =  np.sqrt(s_r_squared)
  # Test score
  z = (sum_run - r_hat) / s_r
  # Get normal table 
  z_alpha = st.norm.ppf(1-alpha)
  # Check hypothesis
  return z, z_alpha,cum_sum,n_p,n_n,sum_run

In [ ]:
def html_beautifulsoup(url):
    response = requests.get(url)
    html = response.content

    #beautifulsoup html will not look like soure
    #review beautifulsoup html to use find and findall
    soup = BeautifulSoup(html,"lxml")
    return soup

#soup_html: soup html is not same as source html
def yahoo_description(soup_html):
    element = soup_html.find('h1',class_='D(ib) Fz(16px) Lh(18px)')
    return element.text  

In [ ]:
def usr_input():
  previousday = dt.date.today()-dt.timedelta(days=1)# get the date of previous day
  Default_end = previousday.strftime("%Y-%m-%d") # change the format of the default datetime 
  years_ago = dt.date.today()-dt.timedelta(days=1852)
  Default_start = years_ago.strftime("%Y-%m-%d")
  Ticker = input('Please enter ticker(Default = JNJ)-->') or 'JNJ'
  Start = input('Enter start date(' + 'Default = 5 Years ago, ' + Default_start + ')-->') or Default_start
  End = input('Enter end date(Default = Yesterday, ' + Default_end + ')-->') or Default_end

  soup_html = html_beautifulsoup("https://finance.yahoo.com/quote/"  + Ticker)
  stock_name = yahoo_description(soup_html)

  return Ticker,Start,End,stock_name

In [ ]:
Ticker,Start,End,stock_name = usr_input()
temp = get_data(Ticker,start_date = Start,end_date=End,index_as_date=True)['adjclose']
Ret = temp.pct_change().dropna()
z,z_alpha,cum,n_p,n_n,run = run_test(Ret.to_numpy(),0)
Ret = Ret.to_frame()
Ret.columns = ['Return']
Ret.insert(1, "runs", cum, True) 

Please enter ticker(Default = JNJ)-->
Enter start date(Default = 5 Years ago, 2015-05-23)-->
Enter end date(Default = Yesterday, 2020-06-16)-->


In [ ]:
pd.set_option('display.max_rows', None)
print(tabulate(Ret,Ret.columns))

                           Return    runs
-------------------  ------------  ------
2015-05-27 00:00:00   0.00307465        0
2015-05-28 00:00:00   0.000197645       0
2015-05-29 00:00:00  -0.0099849         1
2015-06-01 00:00:00  -0.000998694       1
2015-06-02 00:00:00  -0.000599658       1
2015-06-03 00:00:00   0.00109999        2
2015-06-04 00:00:00  -0.00859195        3
2015-06-05 00:00:00  -0.00644998        3
2015-06-08 00:00:00  -0.00639021        3
2015-06-09 00:00:00   0.0025523         4
2015-06-10 00:00:00   0.00682198        4
2015-06-11 00:00:00   0.00364087        4
2015-06-12 00:00:00  -0.00896827        5
2015-06-15 00:00:00  -0.00874431        5
2015-06-16 00:00:00   0.00892423        6
2015-06-17 00:00:00   0.00376163        6
2015-06-18 00:00:00   0.0165097         6
2015-06-19 00:00:00  -0.00498218        7
2015-06-22 00:00:00   0.00230323        8
2015-06-23 00:00:00  -0.00309726        9
2015-06-24 00:00:00  -0.00450981        9
2015-06-25 00:00:00  -0.00211411  

In [ ]:
Ret.to_csv('Return.csv') 
files.download('Return.csv')
print ("On the pc the file was downloaded to your 'Downloads' folder.")

On the pc the file was downloaded to your 'Downloads' folder.


In [ ]:
print('Test result for '+stock_name)
print('')
result = {'n1':[n_p],'n2':[n_p],'run':[run],'statistic':[z],'Z-score':[z_alpha]}
result = pd.DataFrame.from_dict(result)
print(tabulate(result,result.columns))
print(' ')

if z

Test result for JNJ - Johnson & Johnson

      n1    n2    run    statistic    Z-score
--  ----  ----  -----  -----------  ---------
 0   663   663    662      1.43826    1.64485
